## Classifier - Try 11

Classify articles frames using aggregated SRL and sentence embeddings.

1. Try multi attention header for better identifying how each sentence corresponds to the full article frame

In [1]:
import os

os.chdir("../")

In [2]:
labels_path = "data/data/en/train-labels-subtask-2.txt"
articles_path = "data/data/en/train-articles-subtask-2/"

In [3]:
import pandas as pd

# Read the dev-labels-subtask-2.txt file
labels_df = pd.read_csv(labels_path, sep="\t")

# Rename the columns for easier processing
labels_df.columns = ["article_id", "frames"]


labels_df.head()

,article_id,frames
0,832959523,"Morality,Security_and_defense,Policy_prescript..."
1,833039623,"Political,Crime_and_punishment,External_regula..."
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq..."
3,814777937,"Political,Morality,Fairness_and_equality,Exter..."
4,821744708,"Policy_prescription_and_evaluation,Political,L..."


In [4]:
# A function to read the article text given its ID
def get_article_content(article_id):
    try:
        with open(f"{articles_path}/article{article_id}.txt", "r") as f:
            return f.read()
    except FileNotFoundError:
        return None

df = labels_df

# Apply the function to get the article content
df["content"] = df["article_id"].apply(get_article_content)

# Drop rows where content could not be found
df.dropna(subset=["content"], inplace=True)

df.head()


,article_id,frames,content
0,832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n\nThose were the time...
1,833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...
3,814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...
4,821744708,"Policy_prescription_and_evaluation,Political,L...",‘Special place in hell’ for those who promoted...


In [5]:
# Split the frames column into a list of frames
df["frames_list"] = df["frames"].str.split(",")

# create for each frame a new column with the frame as name and 1 if the frame is present in the article and 0 if not
for frame in df["frames_list"].explode().unique():
    df[frame] = df["frames_list"].apply(lambda x: 1 if frame in x else 0)

df.head()

,article_id,frames,content,frames_list,Morality,Security_and_defense,Policy_prescription_and_evaluation,Legality_Constitutionality_and_jurisprudence,Economic,Political,Crime_and_punishment,External_regulation_and_reputation,Public_opinion,Fairness_and_equality,Capacity_and_resources,Quality_of_life,Cultural_identity,Health_and_safety
0,832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n\nThose were the time...,"[Morality, Security_and_defense, Policy_prescr...",1,1,1,1,1,0,0,0,0,0,0,0,0,0
1,833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...,"[Political, Crime_and_punishment, External_reg...",0,0,1,1,0,1,1,1,1,0,0,0,0,0
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...,"[Political, Crime_and_punishment, Fairness_and...",0,0,0,1,0,1,1,1,0,1,0,0,0,0
3,814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...,"[Political, Morality, Fairness_and_equality, E...",1,1,0,0,1,1,0,1,1,1,0,0,0,0
4,821744708,"Policy_prescription_and_evaluation,Political,L...",‘Special place in hell’ for those who promoted...,"[Policy_prescription_and_evaluation, Political...",0,0,1,1,0,1,0,1,0,0,0,0,0,0


In [6]:
filtered_data = df[(df['Political'] == 1) | (df['Crime_and_punishment'] == 1)]

filtered_data.head()

,article_id,frames,content,frames_list,Morality,Security_and_defense,Policy_prescription_and_evaluation,Legality_Constitutionality_and_jurisprudence,Economic,Political,Crime_and_punishment,External_regulation_and_reputation,Public_opinion,Fairness_and_equality,Capacity_and_resources,Quality_of_life,Cultural_identity,Health_and_safety
1,833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...,"[Political, Crime_and_punishment, External_reg...",0,0,1,1,0,1,1,1,1,0,0,0,0,0
2,833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...,"[Political, Crime_and_punishment, Fairness_and...",0,0,0,1,0,1,1,1,0,1,0,0,0,0
3,814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...,"[Political, Morality, Fairness_and_equality, E...",1,1,0,0,1,1,0,1,1,1,0,0,0,0
4,821744708,"Policy_prescription_and_evaluation,Political,L...",‘Special place in hell’ for those who promoted...,"[Policy_prescription_and_evaluation, Political...",0,0,1,1,0,1,0,1,0,0,0,0,0,0
5,833036489,"Political,External_regulation_and_reputation,P...",Bill Maher says he doesn't need Mueller report...,"[Political, External_regulation_and_reputation...",0,0,1,1,0,1,0,1,1,0,0,0,0,0


In [7]:
X = df["content"]
y = df.drop(columns=["article_id", "frames", "frames_list", "content"])

In [8]:
X.head()

0    How Theresa May Botched\n\nThose were the time...
1    Robert Mueller III Rests His Case—Dems NEVER W...
2    Robert Mueller Not Recommending Any More Indic...
3    The Far Right Is Trying to Co-opt the Yellow V...
4    ‘Special place in hell’ for those who promoted...
Name: content, dtype: object

In [9]:
y.head()

,Morality,Security_and_defense,Policy_prescription_and_evaluation,Legality_Constitutionality_and_jurisprudence,Economic,Political,Crime_and_punishment,External_regulation_and_reputation,Public_opinion,Fairness_and_equality,Capacity_and_resources,Quality_of_life,Cultural_identity,Health_and_safety
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,1,1,1,1,0,0,0,0,0
2,0,0,0,1,0,1,1,1,0,1,0,0,0,0
3,1,1,0,0,1,1,0,1,1,1,0,0,0,0
4,0,0,1,1,0,1,0,1,0,0,0,0,0,0


In [10]:
len(y.columns)

14

In [11]:
len(X), len(y)

(432, 432)

# Load Test Data

In [12]:
df_test = pd.read_csv("data/baselines/baseline-output-subtask2-test-en.txt", sep="\t")

df_test.columns = ["article_id", "frames"]

df_test.head()

,article_id,frames
0,311,"External_regulation_and_reputation,Morality,Po..."
1,3132,"External_regulation_and_reputation,Security_an..."
2,3138,"Legality_Constitutionality_and_jurisprudence,P..."
3,3154,Legality_Constitutionality_and_jurisprudence
4,3126,"Political,Security_and_defense"


In [13]:
def get_test_article_content(article_id):
    try:
        with open(f"data/data/en/test-articles-subtask-2/article{article_id}.txt", "r") as f:
            return f.read()
    except FileNotFoundError:
        return None

# Apply the function to get the article content
df_test["content"] = df_test["article_id"].apply(get_test_article_content)

# Drop rows where content could not be found
df_test.dropna(subset=["content"], inplace=True)

df_test = df_test.drop(columns=["frames"])

df_test.head()

,article_id,content
0,311,Journalist names obstacle to peace between Ukr...
1,3132,Putin ally admits Kyiv could soon bomb Moscow ...
2,3138,US debt surpasses $31 trillion\n\nThe US natio...
3,3154,"Oh Thank God, Banksy Is In Ukraine\n\nThe war ..."
4,3126,"U.S., European health officials admit Microsof..."


### Create Dataset

In [14]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='transformers')

### Extract SRL Embeddings from articles

In [15]:
!pip install pycuda
!pip install allennlp allennlp-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 24.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp39-cp39-linux_x86_64.whl size=661947 sha256=9e3f0c8d77dfc6cb7b7ce7a8d196325cdf21a378fa5ee3924307946a14d05ec8
  Stored in directory: /root/.cache/pip/wheels/c8/24/b2/c54ef2a7a3dcb9daa9e403c036360b7e47a8200c94413d0c77
Successfully built pycuda
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 52.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... d

In [16]:
from allennlp.predictors.predictor import Predictor
from allennlp_models.structured_prediction.models import srl_bert
from nltk.tokenize import sent_tokenize
import pandas as pd

In [17]:
from tqdm.notebook import tqdm

def batched_extract_srl_components(sentences, predictor):
    # Prepare the batched input for the predictor
    batched_input = [{'sentence': sentence} for sentence in sentences]
    batched_srl = predictor.predict_batch_json(batched_input)
    
    # Extract SRL components from the batched predictions
    results = []
    for srl in batched_srl:
        best_extracted_data = None
        second_best_extracted_data = None
        for verb_entry in srl['verbs']:
            tags = verb_entry['tags']
            arg0_indices = [i for i, tag in enumerate(tags) if tag in ['B-ARG0', 'I-ARG0']]
            arg1_indices = [i for i, tag in enumerate(tags) if tag in ['B-ARG1', 'I-ARG1']]

            if arg0_indices and arg1_indices:
                best_extracted_data = {
                    'predicate': verb_entry['verb'],
                    'ARG0': ' '.join([srl['words'][i] for i in arg0_indices]),
                    'ARG1': ' '.join([srl['words'][i] for i in arg1_indices])
                }
                break
            elif (arg0_indices or arg1_indices) and not second_best_extracted_data:
                second_best_extracted_data = {
                    'predicate': verb_entry['verb'],
                    'ARG0': ' '.join([srl['words'][i] for i in arg0_indices]) if arg0_indices else '',
                    'ARG1': ' '.join([srl['words'][i] for i in arg1_indices]) if arg1_indices else ''
                }

        if best_extracted_data:
            results.append(best_extracted_data)
        elif second_best_extracted_data:
            results.append(second_best_extracted_data)
            
    return results

def optimized_extract_srl(X, predictor, batch_size=32):
    total_articles = len(X)
    processed_articles = 0

    all_results = []

    for article in X:
        sentences = sent_tokenize(article)
        article_srls = []

        for i in range(0, len(sentences), batch_size):
            batched_sentences = sentences[i:i+batch_size]
            article_srls.extend(batched_extract_srl_components(batched_sentences, predictor))

        all_results.append(article_srls)
        processed_articles += 1
        print(f"Processed article {processed_articles}/{total_articles}")

    return pd.Series(all_results)

In [18]:
import pickle

def get_X_srl(X, recalculate=False, pickle_path="../notebooks/classifier/X_srl_filtered.pkl"):
    """
    Returns the X_srl either by loading from a pickled file or recalculating.
    """
    if recalculate or not os.path.exists(pickle_path):
        print("Recalculate SRL")
        # Load predictor
        predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)
        X_srl = optimized_extract_srl(X, predictor)
        with open(pickle_path, 'wb') as f:
            pickle.dump(X_srl, f)
    else:
        print("Load SRL from Pickle")
        with open(pickle_path, 'rb') as f:
            X_srl = pickle.load(f)
    return X_srl

# GPU

In [19]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

def free_gpu():
    print(torch.cuda.mem_get_info())
    print(torch.cuda.memory_summary())

Using device: cuda


In [20]:
import torch
import gc

def list_gpu_tensors():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    obj = obj.cpu()
                    obj = obj.to("cpu")
                    print(type(obj), obj.size())
        except:
            pass

        
list_gpu_tensors()

/usr/local/lib/python3.9/dist-packages/torch/distributed/distributed_c10d.py:181: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


# Dataset

In [21]:
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd
import nltk

class ArticleDataset(Dataset):
    def __init__(self, X, X_srl, tokenizer, labels=None, max_sentences_per_article=32, max_sentence_length=32, max_arg_length=16):
        self.X = X
        self.X_srl = X_srl
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_sentences_per_article = max_sentences_per_article
        self.max_sentence_length = max_sentence_length
        self.max_arg_length = max_arg_length
        nltk.download('punkt')  # Download the Punkt tokenizer model for sentence splitting
        
    def __len__(self):
        return len(self.X)
    
    def _truncate_or_pad(self, lst, target_length, pad_value=0):
        """
        Truncate or pad the input list to match the target length.
        """
        if len(lst) > target_length:
            return lst[:target_length]
        else:
            return lst + [pad_value] * (target_length - len(lst))
    
    def __getitem__(self, idx):
        article = self.X.iloc[idx]
        srl = self.X_srl.iloc[idx]

        # Split the article into sentences
        sentences = nltk.sent_tokenize(article)
        sentences = sentences[:self.max_sentences_per_article]  # Limit the number of sentences

        # Tokenize and pad/truncate the sentences
        sentence_ids = [self.tokenizer.encode(sentence, add_special_tokens=True, max_length=self.max_sentence_length, truncation=True, padding='max_length') for sentence in sentences]
        while len(sentence_ids) < self.max_sentences_per_article:
            sentence_ids.append([0] * self.max_sentence_length)

        # Tokenize and pad/truncate the SRL items
        predicate_ids = [self.tokenizer.encode(predicate, add_special_tokens=True, max_length=self.max_arg_length, truncation=True, padding='max_length') for predicate in [item['predicate'] for item in srl]]
        arg0_ids = [self.tokenizer.encode(arg0, add_special_tokens=True, max_length=self.max_arg_length, truncation=True, padding='max_length') for arg0 in [item.get('arg0', '') for item in srl]]
        arg1_ids = [self.tokenizer.encode(arg1, add_special_tokens=True, max_length=self.max_arg_length, truncation=True, padding='max_length') for arg1 in [item.get('arg1', '') for item in srl]]
        
        predicate_ids = predicate_ids[:self.max_sentences_per_article]
        arg0_ids = arg0_ids[:self.max_sentences_per_article]
        arg1_ids = arg1_ids[:self.max_sentences_per_article]  
        
        while len(predicate_ids) < self.max_sentences_per_article:
            predicate_ids.append([0] * self.max_arg_length)
        while len(arg0_ids) < self.max_sentences_per_article:
            arg0_ids.append([0] * self.max_arg_length)
        while len(arg1_ids) < self.max_sentences_per_article:
            arg1_ids.append([0] * self.max_arg_length)

        data = {
            'sentence_ids': torch.tensor(sentence_ids, dtype=torch.long),
            'predicate_ids': torch.tensor(predicate_ids, dtype=torch.long),
            'arg0_ids': torch.tensor(arg0_ids, dtype=torch.long),
            'arg1_ids': torch.tensor(arg1_ids, dtype=torch.long)
        }
        
        if self.labels is not None:
            data['labels'] = self.labels.iloc[idx]
        
        return data


In [22]:
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def custom_collate_fn(batch):
    # Extract individual lists from the batch
    sentence_ids = [item['sentence_ids'] for item in batch]
    predicate_ids = [item['predicate_ids'] for item in batch]
    arg0_ids = [item['arg0_ids'] for item in batch]
    arg1_ids = [item['arg1_ids'] for item in batch]
    
    # Pad each list
    sentence_ids = torch.nn.utils.rnn.pad_sequence(sentence_ids, batch_first=True, padding_value=0)
    predicate_ids = torch.nn.utils.rnn.pad_sequence(predicate_ids, batch_first=True, padding_value=0)
    arg0_ids = torch.nn.utils.rnn.pad_sequence(arg0_ids, batch_first=True, padding_value=0)
    arg1_ids = torch.nn.utils.rnn.pad_sequence(arg1_ids, batch_first=True, padding_value=0)

    # Conditionally extract and add labels
    output_dict = {
        'sentence_ids': sentence_ids,
        'predicate_ids': predicate_ids,
        'arg0_ids': arg0_ids,
        'arg1_ids': arg1_ids
    }
    
    if 'labels' in batch[0]:
        labels = [item['labels'] for item in batch]
        output_dict['labels'] = torch.Tensor(labels)

    return output_dict


def get_datasets_dataloaders(X, y, tokenizer, recalculate_srl=False, pickle_path="../notebooks/X_srl_filtered.pkl", batch_size=16, max_sentences_per_article=32, max_sentence_length=32, max_arg_length=16):
    # Get X_srl
    X_srl = get_X_srl(X, recalculate=recalculate_srl, pickle_path=pickle_path)
    
    test_size = 0.1
    
    # Assuming X, X_srl, and y are already defined and have the same number of samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # Calculate class distributions for y_train and y_test in percentage
    train_dist_percent = (y_train.sum() / y_train.shape[0]) * 100
    test_dist_percent = (y_test.sum() / y_test.shape[0]) * 100

    # Create a DataFrame to display them side by side
    dist_comparison = pd.DataFrame({
        'Class': train_dist_percent.index,
        'Train Distribution (%)': train_dist_percent.values,
        'Test Distribution (%)': test_dist_percent.values
    })
    
    print(dist_comparison)
    
    X_srl_train, X_srl_test, _, _ = train_test_split(X_srl, y, test_size=test_size, random_state=42)
    
    # Create the dataset
    train_dataset = ArticleDataset(X_train, X_srl_train, tokenizer, y_train, max_sentences_per_article, max_sentence_length, max_arg_length)
    test_dataset = ArticleDataset(X_test, X_srl_test, tokenizer, y_test, max_sentences_per_article, max_sentence_length, max_arg_length)

    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn, drop_last=True)
    
    print("CREATION DONE")
    return train_dataset, test_dataset, train_dataloader, test_dataloader



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [23]:
!nvidia-smi

Fri Nov 10 11:47:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   45C    P8    35W / 300W |      3MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
def get_article_dataloader(article, tokenizer, batch_size=4):
    X = pd.Series([article])
    y = None  # No labels for this single article
    
    predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)
    # Directly use the optimized_extract_srl function since we don't need to cache for single articles
    X_srl = optimized_extract_srl(X, predictor)
    
    # Create the dataset
    dataset = ArticleDataset(X, X_srl, tokenizer, y)
    
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
    
    return dataloader

In [25]:
def get_test_dataloader(X, tokenizer, batch_size=4):
    y = None
    
    predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz", cuda_device=0)
    # Directly use the optimized_extract_srl function since we don't need to cache for single articles
    X_srl = optimized_extract_srl(X, predictor)
    
    # Create the dataset
    dataset = ArticleDataset(X, X_srl, tokenizer, y)
    
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
    
    return dataloader

# PyTorch Model
The Model consist out of various Layers.

1. SRL_Embedding
2. Autoencoder
3. FRISSLoss
4. Unsupervised
5. Supervised
6. FRISS

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

## 1. SRL_Embeddings

The layer takes tensors of token IDs with the shape [batch_size, max_num_sentences, max_num_tokens] for the sentence, predicates, arg0 and arg1 and returns for each sentence an embedding with shape [batch_size, embedding_dim] for the sentence, predicate, arg0 and arg1. 

The single embedding for the sentence is extracted by taking the [CLS] token embedding. For the predicate, arg0 and arg1 by taking the mean over all word embeddings in this list of tokens. 

> Possible improvements: Better way of extracting the single embedding for predicate, arg0 and arg1.

In [27]:
from transformers import BertModel
import torch.nn as nn
import torch

class SRL_Embeddings(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased"):
        super(SRL_Embeddings, self).__init__()
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        self.embedding_dim = 768  # for bert-base-uncased

    def forward(self, sentence_ids, predicate_ids, arg0_ids, arg1_ids):
        with torch.no_grad():
            # Extract embeddings directly using BERT
            # Adjust dimensions to 2D for BERT input, then reshape back to 3D
            sentence_embeddings_3d = self.bert_model(sentence_ids.view(-1, sentence_ids.size(-1)))[0].view(sentence_ids.size(0), sentence_ids.size(1), -1, self.embedding_dim)
            predicate_embeddings_3d = self.bert_model(predicate_ids.view(-1, predicate_ids.size(-1)))[0].view(predicate_ids.size(0), predicate_ids.size(1), -1, self.embedding_dim)
            arg0_embeddings_3d = self.bert_model(arg0_ids.view(-1, arg0_ids.size(-1)))[0].view(arg0_ids.size(0), arg0_ids.size(1), -1, self.embedding_dim)
            arg1_embeddings_3d = self.bert_model(arg1_ids.view(-1, arg1_ids.size(-1)))[0].view(arg1_ids.size(0), arg1_ids.size(1), -1, self.embedding_dim)

        # Use [CLS] token embedding for sentences
        sentence_embeddings = sentence_embeddings_3d[:, :, 0, :]

        # Average token embeddings for predicates, ARG0, and ARG1
        predicate_embeddings = predicate_embeddings_3d.mean(dim=2)
        arg0_embeddings = arg0_embeddings_3d.mean(dim=2)
        arg1_embeddings = arg1_embeddings_3d.mean(dim=2)
        
        return sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings

# Generate dummy data for the SRL_Embeddings
batch_size = 2
num_sentences = 12
sentence_length = 8
predicate_length = 8
arg0_length = 8
arg1_length = 8

# Dummy data for sentences, predicates, arg0, and arg1
sentence_ids = torch.randint(0, 10000, (batch_size, num_sentences, sentence_length))
predicate_ids = torch.randint(0, 10000, (batch_size, num_sentences, predicate_length))
arg0_ids = torch.randint(0, 10000, (batch_size, num_sentences, arg0_length))
arg1_ids = torch.randint(0, 10000, (batch_size, num_sentences, arg1_length))

srl_embeddings = SRL_Embeddings()

sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = srl_embeddings(sentence_ids, predicate_ids, arg0_ids, arg1_ids)

print(sentence_embeddings.shape, predicate_embeddings.shape, arg0_embeddings.shape, arg1_embeddings.shape)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 12, 768]) torch.Size([2, 12, 768]) torch.Size([2, 12, 768]) torch.Size([2, 12, 768])


## 2. Autoencoder

The layer takes tensors for `v` (size: [batch_size, embedding_dim]), `v_sentence` (size: [batch_size, embedding_dim]), `tau` (type: _float_), and `identifier` (type: _str_). Where `v` is the embedding of either predicate, arg0 or arg1 identified by the `identifier` parameter. The `v_sentence` is the sentence embedding and `tau` defined the tau for annealing the gumpel softmax.

The forward function returns `vhat` (size: [batch_size, embedding_dim]), `dz` (size: [batch_size, embedding_dim]), `gz` (size: [batch_size, embedding_dim]) and `F` (size: [K, embedding_dim]).

- `vhat`: Reconstructed embedding of SRL
- `dz`: Descriptor weights
- `gz`: Gumbel softmax from logits
- `F`: Dictionary

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CombinedAutoencoder(nn.Module):
    def __init__(self, D_w, D_h, K, dropout_prob=0.3):
        super(CombinedAutoencoder, self).__init__()
        
        self.D_h = D_h
        self.K = K
        
        # Shared feed-forward layer for all views
        self.feed_forward_shared = nn.Linear(2 * D_w, D_h)
        
        # Unique feed-forward layers for each view
        self.feed_forward_unique = nn.ModuleDict({
            'a0': nn.Linear(D_h, K),
            'p': nn.Linear(D_h, K),
            'a1': nn.Linear(D_h, K),
        })

        # Initializing F matrices for each view
        self.F_matrices = nn.ParameterDict({
            'a0': nn.Parameter(torch.randn(K, D_w)),
            'p': nn.Parameter(torch.randn(K, D_w)),
            'a1': nn.Parameter(torch.randn(K, D_w)),
        })

        # Additional layers and parameters
        self.dropout = nn.Dropout(dropout_prob)
        self.batch_norm = nn.BatchNorm1d(D_h)
        self.activation = nn.ReLU()

    def gumbel_sigmoid(self, logits, tau: float = 1, hard: bool = False, threshold: float = 0.5):
        gumbels = (
            -torch.empty_like(logits, memory_format=torch.legacy_contiguous_format).exponential_().log()
        )
        gumbels = (logits + gumbels) / tau
        y_soft = gumbels.sigmoid()

        if hard:
            indices = (y_soft > threshold).nonzero(as_tuple=True)
            y_hard = torch.zeros_like(logits, memory_format=torch.legacy_contiguous_format)
            y_hard[indices[0], indices[1]] = 1.0
            ret = y_hard - y_soft.detach() + y_soft
        else:
            ret = y_soft
        return ret
    
    def forward(self, v_p, v_a0, v_a1, v_sentence, tau):
        h_p, h_a0, h_a1 = self.process_through_shared(v_p, v_a0, v_a1, v_sentence)

        logits_p = self.feed_forward_unique['p'](h_p)
        logits_a0 = self.feed_forward_unique['a0'](h_a0)
        logits_a1 = self.feed_forward_unique['a1'](h_a1)
        
        dz_p = torch.sigmoid(logits_p)
        dz_a0 = torch.sigmoid(logits_a0)
        dz_a1 = torch.sigmoid(logits_a1)
        
        gz_p = self.gumbel_sigmoid(dz_p, tau=tau, hard=True)
        gz_a0 = self.gumbel_sigmoid(dz_a0, tau=tau, hard=True)
        gz_a1 = self.gumbel_sigmoid(dz_a1, tau=tau, hard=True)

        vhat_p = torch.matmul(gz_p, self.F_matrices['p'])
        vhat_a0 = torch.matmul(gz_a0, self.F_matrices['a0'])
        vhat_a1 = torch.matmul(gz_a1, self.F_matrices['a1'])

        return {
            "p": {"vhat": vhat_p, "d": dz_p, "g": gz_p, "F": self.F_matrices['p']},
            "a0": {"vhat": vhat_a0, "d": dz_a0, "g": gz_a0, "F": self.F_matrices['a0']},
            "a1": {"vhat": vhat_a1, "d": dz_a1, "g": gz_a1, "F": self.F_matrices['a1']}
        }
        
    def process_through_shared(self, v_p, v_a0, v_a1, v_sentence):
        concatenated_p = torch.cat((v_p, v_sentence), dim=-1)
        concatenated_a0 = torch.cat((v_a0, v_sentence), dim=-1)
        concatenated_a1 = torch.cat((v_a1, v_sentence), dim=-1)
        
        # Concatenate them along the batch dimension for a single pass through the shared layer
        stacked_embeddings = torch.cat([concatenated_p, concatenated_a0, concatenated_a1], dim=0)
        
        #h_shared = self.dropout(stacked_embeddings)
        h_shared = self.feed_forward_shared(stacked_embeddings)
        
        # Splitting them back to individual embeddings
        batch_size = v_p.shape[0]
        h_shared = h_shared.view(3, batch_size, self.D_h)
        
        h_p, h_a0, h_a1 = h_shared[0], h_shared[1], h_shared[2]
        return h_p, h_a0, h_a1

# Mock Data Preparation
D_h = 768
batch_size = 2
embedding_dim = 768
K = 20
tau = 0.9

# Generating mock embeddings for article, predicate, ARG0, ARG1, and their corresponding sentence embeddings
article_embedding = torch.randn(batch_size, embedding_dim)
v_p = torch.randn(batch_size, embedding_dim)
v_a0 = torch.randn(batch_size, embedding_dim)
v_a1 = torch.randn(batch_size, embedding_dim)

# Testing CombinedAutoencoder
autoencoder = CombinedAutoencoder(embedding_dim, D_h, K)
outputs = autoencoder(v_p, v_a0, v_a1, article_embedding, tau)

# Check shapes of the outputs
print("Output shapes:")
for key, value in outputs.items():
    print(f"{key} -> vhat: {value['vhat'].shape}, d: {value['d'].shape}, g: {value['g'].shape}, F: {value['F'].shape}")

Output shapes:
p -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a0 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a1 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])


## 3. FRISSLoss

The layer calculates the unsupervised loss for predicate, arg0 and arg1. 

The forward function takes as input 3 dicts with the parameters `v`, `v_hat`, `g` and `F`. Where `v` is the embedding of the predicate, arg0 or arg1. The `v_hat` (size: [batch_size, embedding_dim]) is the reconstructed embedding for the predicate, arg0 and arg1. The `g` is the gumbel softmax result (size: [batch_size, embedding_dim]). The `F` (size: [K, embedding_dim]) which is the descriptor dictionary.

The layer returns the loss for each batch. So the output is [batch_size].

In [29]:
class FRISSLoss(nn.Module):
    def __init__(self, lambda_orthogonality, M, t):
        super(FRISSLoss, self).__init__()
        
        self.lambda_orthogonality = lambda_orthogonality
        self.M = M
        self.t = t
        self.triplet_loss = nn.TripletMarginLoss(margin=M)

    def contrastive_loss(self, v, vhat, negatives):
        batch_size = vhat.size(0)
        N = negatives.size(1)
        loss = torch.zeros(batch_size, device=v.device)

        # Calculate true distance between reconstructed and real embeddings
        true_distance = self.l2(vhat, v)

        for i in range(N):  # loop over each element in "negatives"
            # Calculate negative distance for current negative embedding
            negative_distance = self.l2(vhat, negatives[:, i])

            # Compute loss based on the provided logic: l2(vhat, v) + 1 + l2(vhat, negative) and clamp to 0 if below 0
            current_loss = 1 + true_distance - negative_distance
            loss += torch.clamp(current_loss, min=0.0)

        # Normalize the total loss by N
        return loss / N

    
    def l2(self, u, v):
            return torch.sqrt(torch.sum((u - v) ** 2, dim=1))
    
    def focal_triplet_loss(self, v, vhat_z, g, F):
        losses = []
        for i in range(F.size(0)):  # Iterate over each negative example
            # For each negative, compute the loss against the anchor and positive
            loss = self.triplet_loss(vhat_z, v, F[i].unsqueeze(0).expand(v.size(0), -1))
            losses.append(loss)

        loss_tensor = torch.stack(losses)  # This will be [20, 2]
        loss = loss_tensor.mean(dim=0).mean()  # First mean over negatives, then mean over the batch
        return loss
    
    def focal_triplet_loss_old(self, v, vhat_z, g, F):
        _, indices = torch.topk(g, self.t, largest=False, dim=1)

        F_t = torch.stack([F[indices[i]] for i in range(g.size(0))])
        
        g_tz = torch.stack([g[i, indices[i]] for i in range(g.size(0))])
                    
        g_t = g_tz / g_tz.sum(dim=1, keepdim=True)
        
        # if division by zero set all nan values to 0
        g_t[torch.isnan(g_t)] = 0
        
        m_t = self.M * ((1 - g_t)**2)

        # Initializing loss
        loss = torch.zeros_like(v[:, 0])
        
        # Iteratively adding to the loss for each negative embedding
        for i in range(self.t):
            current_v_t = F_t[:, i]
            current_m_t = m_t[:, i]
            
            current_loss = current_m_t + self.l2(vhat_z, v) - self.l2(vhat_z, current_v_t)
            
            loss += torch.max(torch.zeros_like(current_loss), current_loss)
             
        # Normalizing
        loss = loss / self.t
        return loss

    def orthogonality_term(self, F, reg=1e-4):
        gram_matrix = torch.mm(F, F.T)  # Compute the Gram matrix F * F^T
        identity_matrix = torch.eye(gram_matrix.size(0), device=gram_matrix.device)  # Create an identity matrix
        ortho_loss = (gram_matrix - identity_matrix).abs().sum()
        return ortho_loss


    def forward(self, p, a0, a1, p_negatives, a0_negatives, a1_negatives):
        # Extract components from dictionary for predicate p
        v_p, vhat_p, d_p, g_p, F_p = p["v"], p["vhat"], p["d"], p["g"], p["F"]
        
        # Extract components from dictionary for ARG0
        v_a0, vhat_a0, d_a0, g_a0, F_a0 = a0["v"], a0["vhat"], a0["d"], a0["g"], a0["F"]

        # Extract components from dictionary for ARG1
        v_a1, vhat_a1, d_a1, g_a1, F_a1 = a1["v"], a1["vhat"], a1["d"], a1["g"], a1["F"]
        
         # Calculate losses for predicate
        Ju_p = self.contrastive_loss(v_p, vhat_p, p_negatives)        
        Jt_p = self.focal_triplet_loss(v_p, vhat_p, g_p, F_p)
        Jz_p = Ju_p + Jt_p + self.lambda_orthogonality * self.orthogonality_term(F_p) ** 2
        #print(Ju_p, Jt_p, self.orthogonality_term(F_p))
        # Calculate losses for ARG0
        Ju_a0 = self.contrastive_loss(v_a0, vhat_a0, a0_negatives)
        Jt_a0 = self.focal_triplet_loss(v_a0, vhat_a0, g_a0, F_a0)
        Jz_a0 = Ju_a0 + Jt_a0 + self.lambda_orthogonality * self.orthogonality_term(F_a0) ** 2
        
        # Calculate losses for ARG1
        Ju_a1 = self.contrastive_loss(v_a1, vhat_a1, a1_negatives)
        Jt_a1 = self.focal_triplet_loss(v_a1, vhat_a1, g_a1, F_a1)
        Jz_a1 = Ju_a1 + Jt_a1 + self.lambda_orthogonality * self.orthogonality_term(F_a1) ** 2
        
        if torch.isnan(Jz_p).any():
            print("Jz_p has nan")
            
        if torch.isnan(Jz_a0).any():
            print("Jz_a0 has nan")
            
        if torch.isnan(Jz_a1).any():
            print("Jz_a1 has nan")
        
        # Aggregate the losses
        loss = Jz_p + Jz_a0 + Jz_a1
        
        return loss


# Mock Data Preparation
batch_size = 2
embedding_dim = 768
K = 15  # Number of frames/descriptors

# Generating mock embeddings for article, predicate, ARG0, ARG1 and their reconstructions
article_embedding = torch.randn(batch_size, embedding_dim)
v_p = torch.randn(batch_size, embedding_dim)
vhat_p = torch.randn(batch_size, embedding_dim)

v_a0 = torch.randn(batch_size, embedding_dim)
vhat_a0 = torch.randn(batch_size, embedding_dim)

v_a1 = torch.randn(batch_size, embedding_dim)
vhat_a1 = torch.randn(batch_size, embedding_dim)

# Generating mock descriptor weights and descriptor matrices for predicate, ARG0, ARG1
d_p = torch.randn(batch_size, K)
d_a0 = torch.randn(batch_size, K)
d_a1 = torch.randn(batch_size, K)

F_p = torch.randn(K, embedding_dim)
F_a0 = torch.randn(K, embedding_dim)
F_a1 = torch.randn(K, embedding_dim)

g_p = torch.randn(batch_size, K)
g_a0 = torch.randn(batch_size, K)
g_a1 = torch.randn(batch_size, K)

# Generating some negative samples (let's assume 5 negative samples per batch entry)
num_negatives = 8
negatives_p = torch.randn(batch_size, num_negatives, embedding_dim)
negatives_a0 = torch.randn(batch_size, num_negatives, embedding_dim)
negatives_a1 = torch.randn(batch_size, num_negatives, embedding_dim)

# Initialize loss function
lambda_orthogonality = 1e-3

t = 8  # Number of descriptors with smallest weights for negative samples
M = t

loss_fn = FRISSLoss(lambda_orthogonality, M, t)

# Organizing inputs into dictionaries
p = {"v": v_p, "vhat": vhat_p, "d": d_p, "g": g_p, "F": F_p}
a0 = {"v": v_a0, "vhat": vhat_a0, "d": d_a0, "g": g_a0, "F": F_a0}
a1 = {"v": v_a1, "vhat": vhat_a1, "d": d_a1, "g": g_a1, "F": F_a1}

loss_fn = FRISSLoss(lambda_orthogonality, M, t)
loss = loss_fn(p, a0, a1, negatives_p, negatives_a0, negatives_a1)
print("FRiSSLoss output:", loss)

FRiSSLoss output: tensor([772874.0625, 772870.6875])


## 4. FRISSUnsupervised

The `FRISSUnsupervised` layer integrates multiple autoencoders and the previously described `FRISSLoss` layer to achieve an unsupervised learning process over the predicates and their arguments.

### Forward Method:

**Inputs**:
1. **v_p**: Embedding of the predicate with size: [batch_size, D_w].
2. **v_a0**: Embedding of the ARG0 (first argument) with size: [batch_size, D_w].
3. **v_a1**: Embedding of the ARG1 (second argument) with size: [batch_size, D_w].
4. **v_article**: Embedding of the article with size: [batch_size, D_w].
5. **negatives**: Tensor containing negative samples with size: [batch_size, num_negatives, D_w].
6. **tau**: A scalar parameter for the Gumbel softmax in the autoencoder.

**Outputs**:
- A dictionary `results` containing:
    - **loss**: A tensor representing the combined unsupervised loss over the batch with size: [batch_size].
    - **p**: Dictionary containing components for the predicate, including reconstructed embedding (`vhat`), descriptor weights (`d`), Gumbel softmax result (`g`), and the descriptor matrix (`F`).
    - **a0**: Same as `p` but for ARG0.
    - **a1**: Same as `p` but for ARG1.

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming you have already defined CombinedAutoencoder and its methods as provided earlier.

class FRISSUnsupervised(nn.Module):
    def __init__(self, D_w, D_h, K, num_frames, lambda_orthogonality, M, t, dropout_prob=0.3):
        super(FRISSUnsupervised, self).__init__()

        self.loss_fn = FRISSLoss(lambda_orthogonality, M, t)      
        
        # Using the CombinedAutoencoder instead of individual Autoencoders
        self.combined_autoencoder = CombinedAutoencoder(D_w, D_h, K, dropout_prob=dropout_prob)

    def forward(self, v_p, v_a0, v_a1, v_sentence, p_negatives, a0_negatives, a1_negatives, tau):
        outputs = self.combined_autoencoder(v_p, v_a0, v_a1, v_sentence, tau)
        
        outputs_p = outputs["p"]
        outputs_p["v"] = v_p
        
        outputs_a0 = outputs["a0"]
        outputs_a0["v"] = v_a0
        
        outputs_a1 = outputs["a1"]
        outputs_a1["v"] = v_a1
        
        loss = self.loss_fn(
            outputs_p,
            outputs_a0, 
            outputs_a1, 
            p_negatives, a0_negatives, a1_negatives
        )

        results = {
            "loss": loss,
            "p": outputs["p"],
            "a0": outputs["a0"],
            "a1": outputs["a1"]
        }
        
        return results

# Mock Data Preparation
D_h = 768
batch_size = 2
embedding_dim = 768
K = 20
num_frames = 15
tau = 0.9
lambda_orthogonality = 0.1  # Placeholder value, please replace with your actual value
M = 7  # Placeholder value, please replace with your actual value
t = 7  # Placeholder value, please replace with your actual value

# Generating mock embeddings for article, predicate, ARG0, ARG1, and their corresponding sentence embeddings
article_embedding = torch.randn(batch_size, embedding_dim)
v_p = torch.randn(batch_size, embedding_dim)
v_a0 = torch.randn(batch_size, embedding_dim)
v_a1 = torch.randn(batch_size, embedding_dim)

# Generating some negative samples (let's assume 5 negative samples per batch entry)
num_negatives = 10
negatives_p = torch.randn(batch_size, num_negatives, embedding_dim)
negatives_a0 = torch.randn(batch_size, num_negatives, embedding_dim)
negatives_a1 = torch.randn(batch_size, num_negatives, embedding_dim)

# Testing FRISSUnsupervised
unsupervised_module = FRISSUnsupervised(embedding_dim, D_h, K, num_frames, lambda_orthogonality, M, t)
results = unsupervised_module(v_p, v_a0, v_a1, article_embedding, negatives_p, negatives_a0, negatives_a1, tau)

# Print the results' shapes for verification
print("Results' Shapes:")
for key, value in results.items():
    if key == "loss":
        print(f"{key}: {value}")
    else:
        print(f"{key} -> vhat: {value['vhat'].shape}, d: {value['d'].shape}, g: {value['g'].shape}, F: {value['F'].shape}")


Results' Shapes:
loss: tensor([1.6623e+08, 1.6623e+08], grad_fn=<AddBackward0>)
p -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a0 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])
a1 -> vhat: torch.Size([2, 768]), d: torch.Size([2, 20]), g: torch.Size([2, 20]), F: torch.Size([20, 768])


## 5. FRISSSupervised

The layer takes the embeddings from the args and the sentence and predicts frames. 

The embeddings for the args are averaged for each arg individually and then averaged on args level. The final embedding is feed into a linear layer and passed through a sigmoid function. 

The sentence embedding is feed into a linear layer and then into a relu function. After again in a linear function and then averaged. The average embeddung is again feed into a linear layer and lastly in a signoid function. 

It returns a span and sentence based prediction of shape [batch_size, num_frames].

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FRISSSupervised(nn.Module):
    def __init__(self, D_w, num_sentences, K, num_frames, dropout_prob=0.3, sentence_heads=8, srl_heads=8):
        super(FRISSSupervised, self).__init__()

        self.D_w = D_w
        
        # Separate MultiheadAttention layers for each SRL type
        #self.attention_s = nn.MultiheadAttention(embed_dim=D_w, num_heads=sentence_heads, dropout=dropout_prob)
        #self.attention_p = nn.MultiheadAttention(embed_dim=K, num_heads=srl_heads, dropout=dropout_prob)
        #self.attention_a0 = nn.MultiheadAttention(embed_dim=K, num_heads=srl_heads, dropout=dropout_prob)
        #self.attention_a1 = nn.MultiheadAttention(embed_dim=K, num_heads=srl_heads, dropout=dropout_prob)
        
        #self.norm_s = nn.LayerNorm(D_w)
        #self.norm_p = nn.LayerNorm(K)
        #self.norm_a0 = nn.LayerNorm(K)
        #self.norm_a1 = nn.LayerNorm(K)
        
        # SRL Classifier for each SRL type
        self.srl_classifier1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(K * 3 * num_sentences, 3 * K),
            nn.BatchNorm1d(3 * K),
            nn.ReLU(),
            nn.Linear(3 * K, 3 * K),
            nn.BatchNorm1d(3 * K),
            nn.Tanh(),
        )
        
        self.srl_classifier2 = nn.Sequential(
            nn.Linear(3 * K, num_frames)
        )
        
        # Sentence-based Classifier
        self.sentence_classifier1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(D_w * num_sentences, 2 * D_w),
            nn.BatchNorm1d(2 * D_w),
            nn.Tanh(),
            nn.Linear(2 * D_w, 2 * D_w),
            nn.BatchNorm1d(2 *  D_w),
            nn.Tanh(),
            nn.Linear(2 * D_w, D_w),
            nn.BatchNorm1d(D_w),
            nn.Tanh(),
        )
        self.sentence_classifier2 = nn.Sequential(
            nn.Linear(D_w, num_frames)
        )
        
        # Unified Classifier (combining SRL and sentence information)
        self.unified_classifier1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear((D_w + 3 * K) * num_sentences, D_w * (num_sentences // 2)),
            nn.BatchNorm1d(D_w * (num_sentences // 2)),
            nn.Tanh(),
            nn.Dropout(dropout_prob),
            nn.Linear(D_w * (num_sentences // 2), D_w),
            nn.BatchNorm1d(D_w),
            nn.Tanh(),
            nn.Linear(D_w, D_w),
            nn.BatchNorm1d(D_w),
            nn.Tanh(),
            nn.Dropout(dropout_prob),
            nn.Linear(D_w, num_frames)
        )
        
    def forward(self, d_p, d_a0, d_a1, vs):
        # SRL Part (Attention and normalization layers are commented out)
        #d_p, _ = self.attention_p(d_p, d_p, d_p)
        #d_a0, _ = self.attention_a0(d_a0, d_a0, d_a0)
        #d_a1, _ = self.attention_a1(d_a1, d_a1, d_a1)

        #d_p = self.norm_p(d_p)
        #d_a0 = self.norm_a0(d_a0)
        #d_a1 = self.norm_a1(d_a1)

        # Concatenate and classify SRL
        srl_cat = torch.cat((d_p, d_a0, d_a1), dim=1)
        
        srl_transformed = self.srl_classifier1(srl_cat)
        srl_pred = self.srl_classifier2(srl_transformed)
        
        # Sentence Part (Attention and normalization layers are commented out)
        #vs, _ = self.attention_s(vs, vs, vs)
        #vs = self.norm_s(vs)

        s_transformed = self.sentence_classifier1(vs)
        sentence_pred = self.sentence_classifier2(s_transformed)
        
        # Combine SRL and sentence predictions
        combined = torch.cat((d_p, d_a0, d_a1, vs), dim=2)
        
        final_output = self.unified_classifier1(combined)
        
        return srl_pred, sentence_pred, final_output


# Mock Data Preparation

batch_size = 2
embedding_dim = 768
num_frames = 15  # Assuming the number of frames is equal to K for simplicity
num_sentences = 32
K = 20

# Generating mock dsz representations for predicate, ARG0, ARG1
d_p = torch.randn(batch_size, num_sentences, K)
d_a0 = torch.randn(batch_size, num_sentences, K)
d_a1 = torch.randn(batch_size, num_sentences, K) 

# Adjusting the num_heads parameter
srl_heads = 4
sentence_heads = 8

# Adjust the mock sentence embeddings shape
vs = torch.randn(batch_size, num_sentences, embedding_dim)

# Initialize and test the supervised module
supervised_module = FRISSSupervised(embedding_dim, num_sentences, K, num_frames, sentence_heads=sentence_heads, srl_heads=srl_heads)

# Forward pass the mock data
yu_hat, ys_hat, combined = supervised_module(d_p, d_a0, d_a1, vs)
yu_hat.shape, ys_hat.shape, combined.shape

(torch.Size([2, 15]), torch.Size([2, 15]), torch.Size([2, 15]))

## 6. FRISS

In [32]:
import torch.nn as nn

class FRISS(nn.Module):
    def __init__(self, embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob=0.3, sentence_heads=8, srl_heads=4, bert_model_name="bert-base-uncased"):
        super(FRISS, self).__init__()
        
        # Aggregation layer replaced with SRL_Embeddings
        self.aggregation = SRL_Embeddings(bert_model_name)
        
        # Unsupervised training module
        self.unsupervised = FRISSUnsupervised(embedding_dim, D_h, K, num_frames, lambda_orthogonality, M, t, dropout_prob=dropout_prob)
        
        # Supervised training module
        self.supervised = FRISSSupervised(embedding_dim, num_sentences, K, num_frames, dropout_prob=dropout_prob, sentence_heads=sentence_heads, srl_heads=srl_heads)
        
    def negative_sampling(self, embeddings, num_negatives=8):
        batch_size, num_sentences, embedding_dim = embeddings.size()
        negatives = []

        for i in range(batch_size):
            # Get all the indices which are not padded (assuming padding is represented by all-zero vectors)
            non_padded_indices = torch.where(torch.any(embeddings[i] != 0, dim=1))[0]

            # Randomly sample negative indices from non-padded embeddings
            negative_indices = non_padded_indices[torch.randint(0, len(non_padded_indices), (num_negatives,))]

            # If there are fewer non-padded embeddings than required negatives, adjust the negative samples
            while len(negative_indices) < num_negatives:
                additional_indices = non_padded_indices[torch.randint(0, len(non_padded_indices), (num_negatives - len(negative_indices),))]
                negative_indices = torch.cat((negative_indices, additional_indices))

            negative_samples = embeddings[i, negative_indices]
            negatives.append(negative_samples)

        return torch.stack(negatives)    
    
    def forward(self, sentence_ids, predicate_ids, arg0_ids, arg1_ids, tau):
        # Convert input IDs to embeddings
        sentence_embeddings, predicate_embeddings, arg0_embeddings, arg1_embeddings = self.aggregation(sentence_ids, predicate_ids, arg0_ids, arg1_ids)
        
        # Handle multiple spans by averaging predictions
        unsupervised_losses = torch.zeros((sentence_embeddings.size(0),), device=sentence_embeddings.device)
        
        # Creating storage for aggregated d tensors
        d_p_list, d_a0_list, d_a1_list = [], [], []
        
        # Process each span
        for span_idx in range(sentence_embeddings.size(1)):
            s_sentence_span = sentence_embeddings[:, span_idx, :]
            v_p_span = predicate_embeddings[:, span_idx, :]
            v_a0_span = arg0_embeddings[:, span_idx, :]
            v_a1_span = arg1_embeddings[:, span_idx, :]
            
            negatives_p = self.negative_sampling(predicate_embeddings)
            negatives_a0 = self.negative_sampling(arg0_embeddings)
            negatives_a1 = self.negative_sampling(arg1_embeddings)
 
            # Feed the embeddings to the unsupervised module
            unsupervised_results = self.unsupervised(v_p_span, v_a0_span, v_a1_span, s_sentence_span, negatives_p, negatives_a0, negatives_a1, tau)                
            unsupervised_losses += unsupervised_results["loss"]
            
            if torch.isnan(unsupervised_results["loss"]).any():
                print("loss is nan")
            
            # Use the vhat (reconstructed embeddings) for supervised predictions
            d_p_list.append(unsupervised_results['p']['d'])
            d_a0_list.append(unsupervised_results['a0']['d'])
            d_a1_list.append(unsupervised_results['a1']['d'])        
        
        # Aggregating across all spans
        d_p_aggregated = torch.stack(d_p_list, dim=1)
        d_a0_aggregated = torch.stack(d_a0_list, dim=1)
        d_a1_aggregated = torch.stack(d_a1_list, dim=1)
        
        span_pred, sentence_pred, combined_pred = self.supervised(d_p_aggregated, d_a0_aggregated, d_a1_aggregated, sentence_embeddings)
        
        if torch.isnan(span_pred).any():
            print("span_pred has nan:", span_pred)
        
        if torch.isnan(sentence_pred).any():
            print("sentence_pred has nan:", sentence_pred)
        
        # Identify valid (non-nan) losses
        valid_losses = ~torch.isnan(unsupervised_losses)

        # Sum only the valid losses
        #unsupervised_loss = unsupervised_losses[valid_losses].sum()
        
        # Take average by summing the valid losses and dividing by num sentences so that padded sentences are also taken in equation
        unsupervised_loss = unsupervised_losses[valid_losses].sum() / sentence_embeddings.shape[1]
        
        return unsupervised_loss, span_pred, sentence_pred, combined_pred


# Set the necessary parameters
batch_size = 2
embedding_dim = 768
K = 14  # Number of frames/descriptors
num_frames = 14  # Assuming the number of frames is equal to K for simplicity
D_h = 512  # Dimension of the hidden representation
lambda_orthogonality = 0.1
M = 8
t = 8
tau = 1.0

# Define some mock token IDs data parameters
max_sentences_per_article = 8
max_sentence_length = 10
num_sentences = max_sentences_per_article

# Generating mock token IDs for predicate, ARG0, ARG1, and their corresponding sentences
# We assume a vocab size of 30522 (standard BERT vocab size) for simplicity.
vocab_size = 30522

sentence_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_sentence_length))
predicate_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_sentence_length))
arg0_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_sentence_length))
arg1_ids = torch.randint(0, vocab_size, (batch_size, max_sentences_per_article, max_sentence_length))

sentence_embeddings = torch.randn(batch_size, max_sentences_per_article, embedding_dim)
predicate_embeddings = torch.randn(batch_size, max_sentences_per_article, embedding_dim)
arg0_embeddings = torch.randn(batch_size, max_sentences_per_article, embedding_dim)
arg1_embeddings = torch.randn(batch_size, max_sentences_per_article, embedding_dim)

srl_heads = 7
sentence_heads = 8

# Initialize the FRISS model
friss_model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K=K, num_frames=num_frames, srl_heads=srl_heads)

# Forward pass the mock data
unsupervised_loss, span_pred, sentence_pred, combined_pred = friss_model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, 1)
unsupervised_loss, span_pred.shape, sentence_pred.shape, combined_pred.shape

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(tensor(1.3014e+08, grad_fn=<DivBackward0>),
 torch.Size([2, 14]),
 torch.Size([2, 14]),
 torch.Size([2, 14]))

# Train Model

The F1-Score (micro-averaged) and Average Precision Score are chosen as primary metrics for evaluating the multi-label classification task due to the following reasons:

1. **F1-Score (Micro)**:
    - The micro-averaged F1-score computes global counts of true positives, false negatives, and false positives. 
    - It provides a balance between precision (the number of correct positive results divided by the number of all positive results) and recall (the number of correct positive results divided by the number of positive results that should have been returned).
    - Given the imbalance in the label distribution observed in the dataset, the micro-averaged F1-score is robust against this imbalance, making it a suitable metric for optimization.

2. **Average Precision Score**:
    - This metric summarizes the precision-recall curve, giving a single value that represents the average of precision values at different recall levels.
    - It's especially valuable when class imbalances exist, as it gives more weight to the positive class (the rarer class in an imbalanced dataset).

Using these metrics will ensure that the model is optimized for a balanced performance across all labels, even if some labels are rarer than others.

In [33]:
import numpy as np
from sklearn.metrics import f1_score, average_precision_score
from math import exp

from torch.optim.lr_scheduler import ReduceLROnPlateau

def train(model, train_dataloader, test_dataloader, optimizer, loss_function, alpha=0.5, num_epochs=10, tau_min=1, tau_decay=0.95, device='cuda', save_path='../notebooks/', save=False):
    tau = 1
    
    metrics = {
        'f1_span_micro': [],
        'f1_sentence_micro': [],
        'f1_combined_micro': [],
        'f1_span_macro': [],
        'f1_sentence_macro': [],        
        'f1_combined_macro': []
    }
    
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
    
    iteration = 0
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        model.train()
        total_loss = 0
        supervised_total_loss = 0
        unsupervised_total_loss = 0
        batch_progress = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Batches", leave=False)
        
        for batch_idx, batch in batch_progress:            
            iteration = iteration + 1
            
            optimizer.zero_grad()

            sentence_ids = batch['sentence_ids'].to(device)
            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)
            labels = batch['labels'].to(device)
            
            assert torch.isnan(sentence_ids).sum() == 0
            assert torch.isnan(predicate_ids).sum() == 0
            assert torch.isnan(arg0_ids).sum() == 0
            assert torch.isnan(arg1_ids).sum() == 0
    
            
            unsupervised_loss, span_logits, sentence_logits, combined_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, tau)
            
            span_loss = 0
            sentence_loss = 0
            with torch.no_grad():
                span_loss = loss_function(span_logits, labels.float())            
                sentence_loss = loss_function(sentence_logits, labels.float())
            
            # supervised_loss = span_loss + sentence_loss
            supervised_loss = loss_function(combined_logits, labels.float())
            # alpha = unsupervised_loss / (supervised_loss + unsupervised_loss) # try dynamic alpha
            
            combined_loss = alpha * supervised_loss + (1-alpha) * unsupervised_loss
            
            # combined_loss = supervised_loss * unsupervised_loss
            
            if torch.isnan(combined_loss):
                print(f"NaN loss detected at epoch {epoch+1}, batch {batch_idx+1}. Stopping...")
                return
        
            combined_loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # After the backward pass
            if any(p.grad is not None and torch.isnan(p.grad).any() for p in model.parameters()):
                print(f"NaN gradients detected at epoch {epoch+1}, batch {batch_idx+1}. Stopping...")
                return
            
            optimizer.step()

            total_loss += combined_loss.item()
            supervised_total_loss += supervised_loss.item()
            unsupervised_total_loss += unsupervised_loss.item()

            batch_progress.set_description(f"Epoch {epoch+1} ({iteration}) Total Loss: {combined_loss.item():.3f}, SRLs: {span_loss:.3f}, Sentence: {sentence_loss:.3f}, CombinedS: {supervised_loss.item():.3f}, Unsupervised: {unsupervised_loss.item():.3f}")
                        
            if save:
                # Log metrics to CSV
                with open(save_path + 'training_metrics.csv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([batch_idx, epoch+1, combined_loss.item(), supervised_loss.item(), unsupervised_loss.item()])

            # Explicitly delete tensors to free up memory
            del sentence_ids, predicate_ids, arg0_ids, arg1_ids, labels, unsupervised_loss
            torch.cuda.empty_cache()

        print(f"Epoch {epoch+1}/{num_epochs}, Combined Loss: {total_loss/len(train_dataloader)}, Supervised Loss: {supervised_total_loss/len(train_dataloader)}, Unsupervised Loss: {unsupervised_total_loss/len(train_dataloader)}")
        
        model.eval()
        
        span_preds = []
        sentence_preds = []
        combined_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in test_dataloader:
                sentence_ids = batch['sentence_ids'].to(device)
                predicate_ids = batch['predicate_ids'].to(device)
                arg0_ids = batch['arg0_ids'].to(device)
                arg1_ids = batch['arg1_ids'].to(device)
                labels = batch['labels'].to(device)
                
                _, span_logits, sentence_logits, combined_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, tau)
                span_pred = (torch.sigmoid(span_logits) > 0.5).float()
                sentence_pred = (torch.sigmoid(sentence_logits) > 0.5).float()
                combined_pred = (torch.sigmoid(combined_logits) > 0.5).float()

                #print("SRL:", span_pred)
                #print("Sentence:", sentence_pred)
                
                span_preds.append(span_pred.cpu().numpy())
                sentence_preds.append(sentence_pred.cpu().numpy())
                combined_preds.append(combined_pred.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

                # Explicitly delete tensors to free up memory
                del sentence_ids, predicate_ids, arg0_ids, arg1_ids, labels, span_logits, sentence_logits, span_pred, sentence_pred, combined_pred
                torch.cuda.empty_cache()

        all_span_preds = np.vstack(span_preds)
        all_sentence_preds = np.vstack(sentence_preds)
        all_combined_preds = np.vstack(combined_preds)
        all_labels = np.vstack(all_labels)

        f1_span_micro = f1_score(all_labels, all_span_preds, average='micro')
        f1_sentence_micro = f1_score(all_labels, all_sentence_preds, average='micro')
        f1_combined_micro = f1_score(all_labels, all_combined_preds, average='micro')
        
        f1_span_macro = f1_score(all_labels, all_span_preds, average='macro')
        f1_sentence_macro = f1_score(all_labels, all_sentence_preds, average='macro')
        f1_combined_macro = f1_score(all_labels, all_combined_preds, average='macro')

        metrics['f1_span_micro'].append(f1_span_micro)
        metrics['f1_sentence_micro'].append(f1_sentence_micro)
        metrics['f1_combined_micro'].append(f1_combined_micro)
        
        metrics['f1_span_macro'].append(f1_span_macro)
        metrics['f1_sentence_macro'].append(f1_sentence_macro)
        metrics['f1_combined_macro'].append(f1_combined_macro)

        print(f"Validation Metrics - micro F1 - Span/Sentence/Combined: {f1_span_micro:.4f}/{f1_sentence_micro:.4f}/{f1_combined_micro:.4f}, macro F1 - Span/Sentence/Combined: {f1_span_macro:.4f}/{f1_sentence_macro:.4f}/{f1_combined_macro:.4f}")
        
        # Anneal tau at the end of the epoch
        tau = max(tau_min, exp(-tau_decay * iteration))
        
        scheduler.step(total_loss)
        
    if save:
        model_save_path = os.path.join(save_path, 'model1.pth')
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved to {model_save_path}")
    
        with open('../notebooks/metrics.json', 'w') as f:
            json.dump(metrics, f)

    return metrics

# Dataset

In [34]:
num_sentences = 32
batch_size = 32

train_dataset, test_dataset, train_dataloader, test_dataloader = get_datasets_dataloaders(X, y, tokenizer, recalculate_srl=False, batch_size=batch_size, max_sentences_per_article=num_sentences, max_sentence_length=64, max_arg_length=12, pickle_path="notebooks/X_srl_full.pkl")

Load SRL from Pickle
                                           Class  Train Distribution (%)  \
0                                       Morality               47.938144   
1                           Security_and_defense               42.783505   
2             Policy_prescription_and_evaluation               14.690722   
3   Legality_Constitutionality_and_jurisprudence               46.391753   
4                                       Economic                6.185567   
5                                      Political               54.381443   
6                           Crime_and_punishment               53.350515   
7             External_regulation_and_reputation               26.804124   
8                                 Public_opinion                5.670103   
9                          Fairness_and_equality               27.577320   
10                        Capacity_and_resources                6.443299   
11                               Quality_of_life               20.3

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CREATION DONE


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Train

In [37]:
def get_friss_model(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, sentence_heads, srl_heads, bert_model_name="bert-base-uncased", load=True, path="", device='cuda'):
    """
    Loads the weights into an instance of the model class from the given path.
    
    Args:
    - model_class (torch.nn.Module): The class of the model (uninitialized).
    - path (str): Path to the saved weights.
    - device (str): Device to load the model on ('cpu' or 'cuda').
    
    Returns:
    - model (torch.nn.Module): Model with weights loaded.
    """

    # Model instantiation
    model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob=dropout_prob, sentence_heads=sentence_heads, srl_heads=srl_heads, bert_model_name=bert_model_name)
    model = model.to(device)
    
    if load:
        assert path != ""
        model.load_state_dict(torch.load(path, map_location=device))
    
    #model.eval()
    return model

In [38]:
torch.set_printoptions(profile="full")

import torch.optim as optim
import json
import csv

from tqdm.notebook import tqdm


# Hyperparameters
embedding_dim = 768
num_frames = 14

D_h = 768 * 2
lambda_orthogonality = 1e-3

K = 14
t = 8
M = 8
tau_min = 0.5
tau_decay = 5e-4

dropout_prob = 0.2

sentence_heads = 8
srl_heads = 7

friss_model_path = "../notebooks/model1.pth"
bert_model_path = "../notebooks/models/fine-tuned-model/"

# Model instantiation
model = get_friss_model(embedding_dim, 
                        D_h, 
                        lambda_orthogonality, 
                        M, 
                        t, 
                        num_sentences, 
                        K, 
                        num_frames, 
                        dropout_prob=dropout_prob, 
                        sentence_heads=sentence_heads, 
                        srl_heads=srl_heads,
                        bert_model_name=bert_model_path,
                        load=False,
                        path=friss_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# LOSS

# Compute the `weight` parameter for each label
label_frequencies = y.mean()
weights = 1 / (label_frequencies + 1e-10)  # Adding a small value to avoid division by zero

# Compute the `pos_weight` parameter
pos_weights = (1 - label_frequencies) / (label_frequencies + 1e-10)

# Convert the computed weights and pos_weights to PyTorch tensors
weights_tensor = torch.tensor(weights.values, dtype=torch.float32).to(device)
pos_weights_tensor = torch.tensor(pos_weights.values, dtype=torch.float32).to(device)

loss_function = nn.BCEWithLogitsLoss(weight=weights_tensor, pos_weight=pos_weights_tensor, reduction="mean")
optimizer = optim.AdamW(model.parameters(), lr=5e-4)#, weight_decay=1e-5)

# Train the model
alpha_value = 0.5
num_epochs_value = 10
# metrics = train(model, train_dataloader, test_dataloader, optimizer, loss_function, tau_min=tau_min, tau_decay=tau_decay, alpha=alpha_value, num_epochs=num_epochs_value, device=device, save=True)

Some weights of the model checkpoint at ../notebooks/models/fine-tuned-model/ were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../notebooks/models/fine-tuned-model/ and are newly initialized: ['bert.pooler.dense.we

# Grid Search

In [ ]:
from itertools import product
import torch.optim as optim
import csv

from tqdm.notebook import tqdm

# Hyperparameters
embedding_dim = 768
num_frames = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def grid_search(train_dataloader, test_dataloader, search_space, num_epochs=10):
    # Store the results for each hyperparameter combination
    results = {}

    # Fixed values for K and num_frames
    K = 14
    num_frames = 14

    # Fixed values for dropout_prob and bert_model_name (adjust if necessary)
    bert_model_name = "../notebooks/models/fine-tuned-model/"

    # Initialize the file to write metrics
    with open("../notebooks/grid_search_metrics.csv", "w", newline='') as csvfile:
        fieldnames = ['combination', 'alpha', 'lr', 'D_h', 'lambda_orthogonality', 'M', 't', 'tau_min', 'tau_decay', 'dropout_prob', 'epoch', 'f1_span_micro', 'f1_span_macro', 'f1_sentence_micro', 'f1_sentence_macro', 'f1_combined_micro', 'f1_combined_macro']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Calculate the total number of combinations
        total_combinations = 1
        for key, values in search_space.items():
            total_combinations *= len(values)

        # Loop through all combinations
        for idx, combination in enumerate(product(*search_space.values())):
            print(f"Training combination {idx + 1}/{total_combinations}: {combination}")

            # Extract hyperparameters from the current combination
            alpha, lr, tau_min, tau_decay, t, D_h, lambda_orthogonality, M, dropout_prob = combination

            # Initialize the model with current hyperparameters
            model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, bert_model_name)
            model.to(device)

            # Define the optimizer
            optimizer = optim.Adam(model.parameters(), lr=lr)

            # Define loss_function if needed (add this if your train function requires it)

            # Train the model with the current hyperparameters
            epoch_metrics = train(model, train_dataloader, test_dataloader, optimizer, loss_function, alpha=alpha, num_epochs=num_epochs, tau_min=tau_min, tau_decay=tau_decay, device=device, save=False)

            # Write the metrics to the CSV file
            for epoch in range(num_epochs):
                f1_span_micro = epoch_metrics['f1_span_micro'][epoch]
                f1_span_macro = epoch_metrics['f1_span_macro'][epoch]
                f1_sentence_micro = epoch_metrics['f1_sentence_micro'][epoch]
                f1_sentence_macro = epoch_metrics['f1_sentence_macro'][epoch]
                f1_combined_micro = epoch_metrics['f1_combined_micro'][epoch]
                f1_combined_macro = epoch_metrics['f1_combined_macro'][epoch]
                row = {
                    'combination': idx,
                    'alpha': alpha,
                    'lr': lr,
                    'D_h': D_h,
                    'lambda_orthogonality': lambda_orthogonality,
                    'M': M,
                    't': t,
                    'tau_min': tau_min,
                    'tau_decay': tau_decay,
                    'dropout_prob': dropout_prob,
                    'epoch': epoch + 1,
                    'f1_span_micro': f1_span_micro,
                    'f1_span_macro': f1_span_macro,
                    'f1_sentence_micro': f1_sentence_micro,
                    'f1_sentence_macro': f1_sentence_macro,
                    'f1_combined_micro': f1_combined_micro,
                    'f1_combined_macro': f1_combined_macro
                }
                writer.writerow(row)
                csvfile.flush()

    return results

search_space = {
    'alpha': [0.5],
    'lr': [1e-5, 2e-5, 5e-4, 1e-3],
    'tau_min': [0.5],
    'tau_decay': [5e-4],
    't': [5, 8, 10, 20],
    'D_h': [768, 768 * 2, 768 // 2, 768 * 3],
    'lambda_orthogonality': [1e-4, 5e-4, 1e-3, 5e-3, 1e-2],
    'M': [5, 8, 10, 20],
    'dropout_rate': [0.1, 0.2, 0.3, 0.5]
}

# Call the grid search function
results = grid_search(train_dataloader, test_dataloader, search_space, 10)
results

Training combination 1/5120: (0.5, 1e-05, 0.5, 0.0005, 5, 768, 0.0001, 5, 0.1)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/10, Combined Loss: 1064118.53125, Supervised Loss: 8.84961978594462, Unsupervised Loss: 2128228.2291666665
Validation Metrics - micro F1 - Span/Sentence/Combined: 0.3265/0.3214/0.4021, macro F1 - Span/Sentence/Combined: 0.1634/0.2665/0.3623


...

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/10, Combined Loss: 1059151.9479166667, Supervised Loss: 8.841436068216959, Unsupervised Loss: 2118295.0208333335
Validation Metrics - micro F1 - Span/Sentence/Combined: 0.4262/0.3731/0.3561, macro F1 - Span/Sentence/Combined: 0.2073/0.3007/0.3382


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 2/10, Combined Loss: 1058002.03125, Supervised Loss: 7.258231083552043, Unsupervised Loss: 2115996.7916666665
Validation Metrics - micro F1 - Span/Sentence/Combined: 0.4262/0.3746/0.3422, macro F1 - Span/Sentence/Combined: 0.2073/0.3054/0.3238


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

# Test model

In [ ]:
def load_model_from_path(path, embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, sentence_heads, srl_heads, device='cuda'):
    """
    Loads the weights into an instance of the model class from the given path.
    
    Args:
    - model_class (torch.nn.Module): The class of the model (uninitialized).
    - path (str): Path to the saved weights.
    - device (str): Device to load the model on ('cpu' or 'cuda').
    
    Returns:
    - model (torch.nn.Module): Model with weights loaded.
    """

    # Model instantiation
    model = FRISS(embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob=dropout_prob, sentence_heads=sentence_heads, srl_heads=srl_heads)
    model = model.to(device)
    
    model.load_state_dict(torch.load(path, map_location=device))
    
    #model.eval()
    return model


In [ ]:
# Hyperparameters
embedding_dim = 768
num_frames = 14

D_h = 768 * 2
lambda_orthogonality = 1e-3

K = 14
t = 8
M = 8
tau_min = 0.5
tau_decay = 5e-4

dropout_prob = 0.1

sentence_heads = 8
srl_heads = 7


model = load_model_from_path('../notebooks/model1.pth', embedding_dim, D_h, lambda_orthogonality, M, t, num_sentences, K, num_frames, dropout_prob, sentence_heads, srl_heads)

In [ ]:
def predict(model, dataloader, y_columns, device='cuda'):
    """
    Make predictions with the given model and dataloader.
    
    Args:
    - model (torch.nn.Module): The model to make predictions with.
    - dataloader (DataLoader): DataLoader for the dataset to predict on.
    - y_columns (pandas.Index): Column names from the y dataframe which correspond to labels.
    - device (str): Device to make predictions on ('cpu' or 'cuda').
    
    Returns:
    - predicted_labels (list of lists): List containing the predicted labels for each instance.
    """
    model.eval()
    all_preds_span = []
    
    with torch.no_grad():
        for batch in dataloader:
            # Move data to device
            sentence_ids = batch['sentence_ids'].to(device)
            predicate_ids = batch['predicate_ids'].to(device)
            arg0_ids = batch['arg0_ids'].to(device)
            arg1_ids = batch['arg1_ids'].to(device)
            
            # Forward pass
            _, span_logits, sentence_logits, combined_logits = model(sentence_ids, predicate_ids, arg0_ids, arg1_ids, 0.6)
            combined_pred = (torch.sigmoid(combined_logits) > 0.5).float()

            all_preds_span.append(combined_pred.cpu().numpy())
                
            torch.cuda.empty_cache()

    predictions = np.vstack(all_preds_span)
    
    # Convert boolean predictions to labels
    predicted_labels = []
    for pred in predictions:
        labels = list(y_columns[pred.astype(bool)])
        predicted_labels.append(labels)
    
    return predicted_labels


In [ ]:
import numpy as np

# article813452859
article = """Sadiq Khan Slammed for Pro-EU 'Message of Support' During Firework Display

The spectacular fireworks that lit up the London sky on Monday night caused a stir on social media over the display's pro-EU message, at a time when the nation is divided over its looming withdrawal from the bloc.
London Mayor Sadiq Khan faced mounting criticism after the capital's New Year's Eve fireworks display, which celebrated ties with the European Union, left a bad taste in the mouths of some Brits.
The 135-metre-high London Eye was lit up in blue while its tubs turned yellow, with the giant Ferris wheel resembling the star-studded flag of the European Union.Sadiq Khan called his fireworks display a "message of support" to EU citizens living in London.
"Our one million EU citizens are Londoners, they make a huge contribution, and no matter the outcome of Brexit — they will always be welcome", he said.
To the one million EU citizens who have made our city your home: you are Londoners, you make a huge contribution and you are welcome here.
I'm proud that tonight we will welcome in the new year with a message of support to you.
#LondonNYE #LondonIsOpen https://t.co/XctrgfXXaM — Sadiq Khan (@SadiqKhan) 31 декабря 2018 г.
However, a host of Londoners rushed to Twitter to accuse their mayor of "politicising" the celebrations — with some are even calling for his resignation.
I cannot believe this event has been politicised.
This man has no shame.
Just resign.
— wayne campbell (@campbs177) 31 декабря 2018 г.
Thanks a lot Sadiq Khan you ruined the fireworks display by talking about Europe, need I remind you about Brexit.
You have started of the new year by talking about relationships with the European Union.
Well done.
We need Boris Johnson back.
— Mitchell T Cannon (@MitchellTCanno1) 1 января 2019 г.
Another shameless attempt at using party politics on what is supposed to be a happy occasion — droneguy (@shelbyguitars) 1 января 2019 г.
Politicising another innocent event that should be no different to anyone no matter who they are or where they are from!
Shameful!
!
— Mike Dyer (@Miked2372Mike) 31 декабря 2018 г.
Someone was stabbed down the road from me last night.
How about sorting that stuff out instead of politicizing something that should be fun for everyone?How many times does it have to be said.
Commenting on Brexit isn't your job.
— Peter Rockett (@rockettp) 31 декабря 2018 г.
The UK voted to leave the EU in June 2016 via a nationwide referendum, with 51.9 per cent voting in favour of pulling out of the bloc, while 48.1 per cent wanted to remain.
The withdrawal is scheduled for the end of March; the Article 50 deadline.
The Remain sentiment dominated London, with nearly 60 percent of voters wanting Britain to stay in the European Union.
Sadiq Khan, an outspoken Remainer himself, earlier called for a second referendum on Brexit.
"The government's abject failure — and the huge risk we face of a bad deal or a 'no deal' Brexit — means that giving people a fresh say is now the right — and only — approach left for our country," he said in September.
"""

test_article = get_article_dataloader(article, tokenizer)
predict(model, test_article, y.columns)

# Run test for validation

In [ ]:
test_dataloader = get_test_dataloader(df_test["content"], tokenizer)

In [ ]:
predictions = predict(model, test_dataloader, y.columns)

In [ ]:
df_preds = pd.DataFrame(predictions)

In [ ]:
df_preds = pd.concat([df_test, df_preds], axis=1)

In [ ]:
df_preds["pred_frames"] = df_preds.apply(lambda l: list([l[0], l[1], l[2], l[3], l[4], l[5], l[6], l[7]]), axis=1)

df_preds["pred_frames"] = df_preds["pred_frames"].apply(lambda l: ",".join([ f for f in l if f is not None]))

In [ ]:
df_preds.to_csv("../notebooks/test.csv", sep="\t", index=False, columns=["article_id", "pred_frames"])